![](../images/rivacon_frontmark_combined_header.png)

# American plain vanilla option

In [ ]:
import pyvacon.analytics as analytics
import math
import numpy
import pandas as pd
import pyvacon.marketdata.plot as mkt_plot #import module for plotting functionality
#the next lin is a jupyter internal command to show the matplotlib graphs within the notebook
%matplotlib inline
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True) 

In [ ]:
def exp(x):
    return math.exp(x)
def sqrt(x):
    return math.sqrt(x)

## Definition of an American plain vanilla option

In contrast to an European plain vanilla option, American plain vanilla options can be exercised at any time before expiry which gives the holder of the option more rights than in the case of an European option. As a consequence, American options can never be less valuable than European options. In order to price an American options and incorporate the possibility of an early exercise, we need to be familiar with the concept of binomial trees which was described in the [binomial tree](binomial_trees.ipynb) notebook.

When pricing American options, the parameters $u$ and $d$ should match the volatility of the stock price (see [Hull](options-futures-and-other-derivatives-8th-john.pdf), p.265 ff.). The parameters for matching volatility $u$ and $d$ are defined as follows:

$$u = e^{\sigma\sqrt{\Delta{t}}}$$
$$d = e^{-\sigma\sqrt{\Delta{t}}}$$

with $\sigma$ as the standard deviation of the underlying's stock and $\Delta{t}$ as a small time step. 
The probability of an upward movement $p$ is defined as 

$$p=\frac{a-d}{u-d}$$

with

$$a=e^{r\Delta{t}}.$$


## Pricing an American plain vanilla option

The code for the pricing an American plain vanilla option on a non-dividend paying stock using a binomial tree is shown in the following code cell. 

In [ ]:
def AmericanPlainVanillaOption(_type, F0, K, r_f, r_b, r_dsc, sigma, N, T, array=False):
    dt = T/N
    a = exp((r_f-r_b)*dt)
    u = exp(sigma*sqrt(dt))
    d = 1/u
    p = (a-d)/(u-d) # Probability for an upward movement of the stock price
    
    #Price tree
    price_tree = numpy.zeros([N+1, N+1]) # Creating an array for the binomial tree
    
    for i in range(N+1):
        for j in range(i+1):
            price_tree[j, i] = F0*(d**j)*(u**(i-j)) ## Calculating the paths for the stock price
       
    # Option value  (payoff tree)
    option = numpy.zeros([N+1, N+1])
    if _type =='CALL':
        # Calculating the payoff at maturity, i.e. calculating (S_T-K) for each final stock price (last column)
        option[:, N] = numpy.maximum(numpy.zeros(N+1), price_tree[:, N]-K) 
    if _type =='PUT':
        option[:, N] = numpy.maximum(numpy.zeros(N+1), K-price_tree[:, N])
    
    # Calculate option price at t=0
    # Calculating the option prices at the different nodes going backward from the last nodes
    for i in numpy.arange(N-1, -1, -1):
        for j in numpy.arange(0, i+1):
            option[j, i] = exp(-r_dsc*dt)*(p*option[j, i+1]+(1-p)*option[j+1, i+1])
            
    #Return
    # if array: return also price tree and option price tree
    if array:
        return [option[0,0], price_tree, option]
    # otherwise return only option price at t=0
    else: 
        return option[0,0]

In [ ]:
S0 = 50
K = 50
r_dsc = 0.05
r_f = 0.01
r_b = 0.00
sigma = 0.3
N = 2
T = 1
t=0
F0 = F0 = S0*exp((r_f-r_b)*(T-t))

AmericanPlainVanillaOption('CALL',F0, K, r_f, r_b, r_dsc, sigma, N, T)

## Plotting the option price

In [ ]:
spots = []
#strikes = [0,10,20,30,40,50,60,70,80,90,100]
#ttm = [0.1, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 2.5, 3.0, 4.0, 5.0]
#vols = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]

n=0.5
while n<=100:
    spots.append(n)
    n=n+0.1
    
call_prices = []
for i in range(len(spots)):
    call_prices.append(AmericanPlainVanillaOption('CALL', spots[i], K, r_f, r_b, r_dsc, sigma, N, T))
    
put_prices = []
for i in range(len(spots)):
    put_prices.append(AmericanPlainVanillaOption('PUT', spots[i], K, r_f, r_b, r_dsc, sigma, N, T))

#Plot the prices
prices_list = {'Spots': spots, 
              'Call Prices': call_prices,
              'Put Prices': put_prices}

prices = pd.DataFrame(prices_list, index = spots)    

prices_plot_data = [Scatter(name = 'Call prices', x = spots, y = prices_list['Call Prices'], mode = 'lines'),
                Scatter(name = 'Put prices', x = spots, y = prices_list['Put Prices'], mode = 'lines'), 
                Scatter(name = 'Strike', x = [K,K], y = [0,max(call_prices)], mode = 'lines')]

layout = Layout(title = "American Plain Vanilla Option",
               xaxis=dict(title ='Spot'),
               yaxis=dict(title = 'Prices'))

iplot(Figure(data=prices_plot_data, layout=layout))